<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [15]:
import numpy as np
import pandas as pd
from gensim.models.fasttext import FastText
from keras.preprocessing.text import text_to_word_sequence

In [5]:
df = pd.read_csv('../data/train/chats_train.csv')

In [6]:
df.head()

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp
0,1129149026,Pareto Network - Discussion,[],Bezant,526106862,False,None,None,Admin disputes in private thank you.,1524284160
1,1129149026,Pareto Network - Discussion,[],Sheriff,538526216,False,None,None,I see Binance add coins all the time on the Cr...,1524284160
2,1146170349,Binance English,[],DaySandBox,400753764,True,None,daysandbox_bot,Removed msg from __Megzon Kadrija__. Reason: n...,1524285614
3,1146170349,Binance English,[],Kamalendu,435609627,False,Pal,None,Bc xvg dead,1524285372
4,1146170349,Binance English,[':thumbsup:'],Abdelbari,154857846,False,K,None,"Yes, youre right 👍",1524286342


In [7]:
df['text'].fillna('', inplace=True)

In [8]:
df['text'] = df['text'].apply(lambda x: x.lower())

In [9]:
df = df[~df['text'].isnull()].copy()

In [10]:
class SentenceIterator(object):
    texts = None

    def __init__(self, texts):
        self.texts = texts

    def __iter__(self):
        for text in self.texts:
            yield text_to_word_sequence(text, lower=False)

In [11]:
fasttext_model = FastText(SentenceIterator(df['text']), min_count=2)

In [12]:
fasttext_model.save('../data/fasttext_chats.bin')

# to vectors

In [13]:
df = pd.read_csv('../data/train/chats_train.csv')
df['text'].fillna('', inplace=True)
df['text'] = df['text'].apply(lambda x: x.lower())

In [14]:
def text2vec(text):
    vectors = []
    for word in text_to_word_sequence(text):
        try:
            v = fasttext_model.wv[word]
            vectors.append(v)
        except Exception as e:
            pass
    if not vectors:
        return np.zeros((100, ))
    return sum(vectors) / len(vectors)

In [18]:
vectors = list(map(text2vec, df['text']))

In [19]:
vectors = np.array(vectors)

In [22]:
np.save('../data/chat_vectors_100.npy', vectors)